In [13]:
import pandas as pd
import numpy as np
import openpyxl
from scipy.interpolate import griddata


# Load the dataset
file_path = 'Compressor_clean.xlsx'
df = pd.read_excel(file_path, sheet_name=None)

print(df.keys())

print(df)


dict_keys(['page-1_table-1'])
{'page-1_table-1':     Unnamed: 0 Unnamed: 1     36     48     55     70     87    107     130  \
0        613.0          C    NaN    NaN    NaN    NaN    NaN    NaN  34,300   
1          NaN          P    NaN    NaN    NaN    NaN    NaN    NaN   5,230   
2          NaN          A    NaN    NaN    NaN    NaN    NaN    NaN    23.3   
3          NaN          M    NaN    NaN    NaN    NaN    NaN    NaN     594   
4          NaN          E    NaN    NaN    NaN    NaN    NaN    NaN     6.6   
..         ...        ...    ...    ...    ...    ...    ...    ...     ...   
57         NaN          P  2,200  2,190  2,170  2,140  2,080  2,020   1,960   
58         NaN          A    9.9    9.8    9.8    9.6    9.4    9.1     8.8   
59         NaN          M    209    266    298    368    450    544     654   
60         NaN          E    8.0   10.3   11.7   14.9   18.9   23.8    29.8   
61         NaN          %   57.4   63.3   65.7   69.6   71.5   70.7    66.2   

  

In [1]:
import pandas as pd
from scipy.interpolate import interp1d, interp2d
from scipy.interpolate import griddata
import numpy as np

# Load the dataset
file_path = 'Compressor_clean.xlsx'
df = pd.read_excel(file_path)

# Manually set the header based on the correct content
df.columns = ['Condensing_Temperature', 'Type', 36, 48, 55, 70, 87, 107, 130, 143, 156]

# Filter the Capacity (C) data
capacity_data = df[df['Type'] == 'C']
capacity_data = capacity_data.drop(columns=['Type']).reset_index(drop=True)

# Convert the data to numeric, handling the conversion of commas in the values
capacity_data = capacity_data.replace({',': ''}, regex=True).apply(pd.to_numeric, errors='coerce')

# Prepare data for 2D interpolation
condensing_temps = capacity_data['Condensing_Temperature'].values
evap_temps = capacity_data.columns[1:].astype(float)
capacities = capacity_data.iloc[:, 1:].values

# Create a 2D interpolation function
interp_func_2d = interp2d(evap_temps, condensing_temps, capacities, kind='linear')

# Function to interpolate Capacity values
def interpolate_capacity(evap_temp, cond_temp):
    # Check if the evaporating temperature is within the range of available data
    if evap_temp < evap_temps.min() or evap_temp > evap_temps.max():
        return np.nan

    # Check if the condensing temperature is within the range of available data
    if cond_temp < condensing_temps.min() or cond_temp > condensing_temps.max():
        return np.nan

    # Perform 2D interpolation
    interpolated_capacity = interp_func_2d(evap_temp, cond_temp)
    
    return interpolated_capacity[0]  # Extract the interpolated value

evap_temp_input = 140
cond_temp_input = 236
interpolated_capacity = interpolate_capacity(evap_temp_input, cond_temp_input)
print(interpolated_capacity)


62661.53846153847


C:\Users\bober\AppData\Local\Temp\ipykernel_4304\2487454271.py:26: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  interp_func_2d = interp2d(evap_temps, condensing_temps, capacities, kind='linear')
C:\Users\bober\AppData\Local\Temp\ipykernel_4304\2487454271.py:39: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids u

In [26]:
import pandas as pd
from scipy.interpolate import griddata
import numpy as np

# Load the dataset
file_path = 'Compressor_clean.xlsx'
df = pd.read_excel(file_path)

# Manually set the header based on the correct content
df.columns = ['Condensing_Temperature', 'Type', 36, 48, 55, 70, 87, 107, 130, 143, 156]

# Filter the Capacity (C) data
capacity_data = df[df['Type'] == 'C']
capacity_data = capacity_data.drop(columns=['Type']).reset_index(drop=True)

# Convert the data to numeric, handling the conversion of commas in the values
capacity_data = capacity_data.replace({',': ''}, regex=True).apply(pd.to_numeric, errors='coerce')

# Prepare data for 2D interpolation
condensing_temps = capacity_data['Condensing_Temperature'].values
evap_temps = capacity_data.columns[1:].astype(float)
capacities = capacity_data.iloc[:, 1:].values

# Flatten the data for griddata interpolation
points = np.array([(et, ct) for ct in condensing_temps for et in evap_temps])
values = capacities.flatten()

def interpolate_capacity(evap_temp, cond_temp):
    # Perform 2D interpolation
    interpolated_capacity = griddata(points, values, (evap_temp, cond_temp), method='linear')
    
    return interpolated_capacity

# Example usage
evap_temp_input = 107
cond_temp_input = 477
interpolated_capacity = interpolate_capacity(evap_temp_input, cond_temp_input)
print(interpolated_capacity)



33900.0


In [37]:
import pandas as pd
from scipy.interpolate import griddata
import numpy as np

# Load the dataset without headers
file_path = 'Compressor_clean.xlsx'
df = pd.read_excel(file_path, header=None)

# Set the proper headers
headers = df.iloc[0].values
headers[0] = 'Condensing_Temperature'
headers[1] = 'Type'
df.columns = headers

# Remove the header row and reset index
df = df.iloc[1:].reset_index(drop=True)

# Convert numeric columns to numeric type, handling commas
df.iloc[:, 2:] = df.iloc[:, 2:].replace({',': ''}, regex=True).apply(pd.to_numeric, errors='coerce')

# Forward fill the NaN values in the 'Condensing_Temperature' and 'Type' columns
df['Condensing_Temperature'] = df['Condensing_Temperature'].ffill()
df['Type'] = df['Type'].ffill()

# Drop any remaining rows with NaN values in the numerical columns
df = df.dropna()

# Function to interpolate values
def interpolate_data(evap_temp, cond_temp, value_type):
    # Filter the data based on the selected type
    data = df[df['Type'] == value_type]
    data = data.drop(columns=['Type']).reset_index(drop=True)

    # Prepare data for 2D interpolation
    condensing_temps = data['Condensing_Temperature'].values
    evap_temps = data.columns[1:].astype(float)
    values = data.iloc[:, 1:].values

# Flatten the data for griddata interpolation
    points = np.array([(et, ct) for ct in condensing_temps for et in evap_temps])
    values = values.flatten()

# Ensure that values are numeric
    values = pd.to_numeric(values, errors='coerce')

# Remove any points with NaN values
    valid_mask = ~np.isnan(values)
    points = points[valid_mask]
    values = values[valid_mask]

    # Perform 2D interpolation
    interpolated_value = griddata(points, values, (evap_temp, cond_temp), method='linear')
    
    return interpolated_value

# Get the ranges for Condensing_Temperature and evaporation temperatures
condensing_temp_range = (df['Condensing_Temperature'].min(), df['Condensing_Temperature'].max())
evap_temps_range = (df.columns[2:].astype(float).min(), df.columns[2:].astype(float).max())

print(f"Condensing Temperature Range: {condensing_temp_range}")
print(f"Evaporation Temperature Range: {evap_temps_range}")

evap_temp_input = 156  # within the range 36.0 to 156.0
cond_temp_input = 237  # within the range 236.0 to 318.0
value_types = ['C', 'P', 'A', 'M', 'E', '%']  # All value types

# Interpolate and print values for all types
results_within_range = {}
for value_type in value_types:
    interpolated_value = interpolate_data(evap_temp_input, cond_temp_input, value_type)
    results_within_range[value_type] = interpolated_value
    print(f"Interpolated value for {value_type}: {interpolated_value}")

results_within_range


Condensing Temperature Range: (236.0, 318.0)
Evaporation Temperature Range: (36.0, 156.0)
Interpolated value for C: 70007.8947368421
Interpolated value for P: 1902.2368421052633
Interpolated value for A: 8.631578947368421
Interpolated value for M: 779.8684210526317
Interpolated value for E: 36.83421052631579
Interpolated value for %: 56.794736842105266


{'C': array(70007.89473684),
 'P': array(1902.23684211),
 'A': array(8.63157895),
 'M': array(779.86842105),
 'E': array(36.83421053),
 '%': array(56.79473684)}